In [1]:
import os, sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
import database
import pandas as pd
from snowballing.operations import reload, load_work_map_all_years
reload()
from snowballing.strategies import Strategy
strategy = Strategy({})

def pages(work):
    pp = getattr(work, 'pp', '')
    if not pp:
        return 0
    split = list(map(int, pp.split('--')))
    if len(split) == 1:
        return split[0]
    return split[-1] - split[0] + 1

In [2]:
selected = sorted({
    w
    for _, w in load_work_map_all_years()
    if w.place.acronym == "IPAW"
    if pages(w) >= 6
    if w.year >= 2008
}, key=lambda w: int(w.metakey[-5:-1]))

In [3]:
len(selected)

98

In [4]:
places = {
    w.place
    for _, w in load_work_map_all_years()
}

In [5]:
len(places)

341

In [6]:
{
    p.type for p in places
}

{'Archive',
 'Book',
 'Conference',
 'Journal',
 'Lang',
 'Magazine',
 'Patent',
 'Peer Reviewed Book',
 'Tech Report',
 'Thesis'}

In [7]:
result = []

for w in selected:
    scholar = int(getattr(w, "scholar_count", 0))
    actual = len(set(strategy.rev_ref[w]))
    books = len({r for r in strategy.rev_ref[w]
                 if r.place.type == "Book"})
    thesis = len({r for r in strategy.rev_ref[w]
                  if r.place.type == "Thesis"})
    lang = len({r for r in strategy.rev_ref[w]
                if r.place.type == "Lang"})
    other = len({r for r in strategy.rev_ref[w]
                 if r.place.type in (
                    "Archive", "Magazine", "Patent",
                    "Tech Report"
                 )})
    result.append([
        w.metakey, scholar, actual, books, thesis, lang, other,
        actual - books,
        actual - thesis - books,
        actual - thesis - books - lang - other,
    ])
    
header = [
    "ID", "Scholar", "Without Repetition", 
    "Books", "Thesis/Dissertation/Monograph (TDM)",
    "Unindentified Languages",
    "Other Grey Literature",
    "Without Books",
    "Without Books and TDM",
    "Only Conference and Journal papers"
]
df = pd.DataFrame(result, columns=header)
df

,ID,Scholar,Without Repetition,Books,Thesis/Dissertation/Monograph (TDM),Unindentified Languages,Other Grey Literature,Without Books,Without Books and TDM,Only Conference and Journal papers
0,park2008a,40,40,3,4,1,4,37,33,28
1,kwasnikowska2008a,24,24,0,6,2,1,24,18,15
2,nagappan2008a,9,9,0,1,0,0,9,8,8
3,chen2008a,6,6,1,0,1,0,5,5,4
4,bochner2008a,17,16,1,5,0,1,15,10,9
5,callahan2008a,27,27,2,3,0,1,25,22,21
6,oliveira2008a,34,33,0,7,1,2,33,26,23
7,bowers2008a,77,75,6,11,6,5,69,58,47
8,crawl2008a,64,62,4,11,0,5,58,47,42
9,misra2008a,37,36,2,5,0,7,34,29,22


In [50]:
def hindex(result, index, category):
    sort_by_index = [
        (x[0], x[index])
        for x in sorted(result, reverse=True, key=lambda x: x[index])
    ]
    hindex_list = [
        x for i, x in enumerate(sort_by_index)
        if x[1] >= i + 1
    ]
    print(f"{category}: {len(hindex_list)}")
    df = pd.DataFrame(sort_by_index, columns=["ID", "Count"])
    df.index += 1 
    return df.style.apply(
        lambda s: ['background-color: #FF8888' if s.name > s.Count else "" for _ in s], axis=1
    )

In [52]:
category = "Scholar"
index = 1
hindex(result, index, category)

Scholar: 20


,ID,Count
1,hartig2010a,119
2,bowers2008a,77
3,missier2008a,76
4,missier2010a,73
5,murta2014a,67
6,crawl2008a,64
7,santos2008a,41
8,park2008a,40
9,misra2008a,37
10,oliveira2008a,34


In [53]:
category = "Without Repetitions"
index = 2
hindex(result, index, category)

Without Repetitions: 19


,ID,Count
1,hartig2010a,116
2,bowers2008a,75
3,missier2008a,73
4,missier2010a,69
5,murta2014a,66
6,crawl2008a,62
7,park2008a,40
8,santos2008a,40
9,misra2008a,36
10,oliveira2008a,33


In [54]:
category = "Without Books"
index = 7
hindex(result, index, category)

Without Books: 19


,ID,Count
1,hartig2010a,108
2,bowers2008a,69
3,missier2008a,67
4,missier2010a,65
5,murta2014a,60
6,crawl2008a,58
7,santos2008a,39
8,park2008a,37
9,misra2008a,34
10,oliveira2008a,33


In [55]:
category = "Without Books, Theses, Dissertations, Monographs"
index = 8
hindex(result, index, category)

Without Books, Theses, Dissertations, Monographs: 18


,ID,Count
1,hartig2010a,90
2,missier2008a,60
3,bowers2008a,58
4,missier2010a,56
5,murta2014a,50
6,crawl2008a,47
7,park2008a,33
8,santos2008a,31
9,misra2008a,29
10,oliveira2008a,26


In [56]:
category = "Without Grey literature:"
index = 9
hindex(result, index, category)

Without Grey literature:: 17


,ID,Count
1,hartig2010a,79
2,missier2008a,54
3,bowers2008a,47
4,missier2010a,47
5,crawl2008a,42
6,murta2014a,42
7,santos2008a,29
8,park2008a,28
9,altintas2010a,24
10,oliveira2008a,23
